In [3]:
# Import các thư viện cần thiết
import numpy as np
import cv2
import pandas as pd
import os
import pickle
from google.colab import drive
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split

In [4]:
# Connect Google Drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/TGMT/'
# Hiển thị tất cả thư mục hoặc file
os.listdir(path)

Mounted at /content/drive


['Models', 'Data', 'imgface.jpg', 'data_face_features.pickle']

In [5]:
# Load data from pickle file
data = pickle.load(open("/content/drive/MyDrive/TGMT/data_face_features.pickle", mode="rb"))

In [6]:
x = np.array(data['data'])
y = np.array(data['label'])

print(x)
print(y)

[[[ 4.12972440e-04  2.00944394e-01 -7.10193635e-05 ... -6.53045923e-02
    7.79734850e-02  6.92033395e-03]]

 [[ 2.41263434e-02  2.04540446e-01 -7.79361743e-03 ...  2.65857968e-02
    1.21704385e-01  1.49168121e-02]]

 [[ 1.27565507e-02  2.20630527e-01 -1.08428365e-02 ...  9.52625833e-03
    1.50048286e-01  2.36851294e-02]]

 ...

 [[ 1.01158537e-01  2.66881257e-01 -5.40876798e-02 ... -8.94514769e-02
    1.64200380e-01  2.83455960e-02]]

 [[ 1.29780397e-01  9.85381305e-02 -7.15201050e-02 ... -2.76106559e-02
    8.41911510e-02 -7.54967192e-03]]

 [[ 6.43560067e-02  6.15202682e-03 -4.06341329e-02 ...  1.12314321e-01
    6.11688420e-02  5.67569509e-02]]]
['1721031620_PhanQuocHuy' '1721031620_PhanQuocHuy'
 '1721031620_PhanQuocHuy' ... '162000186_NguyenVanHuong'
 '162000186_NguyenVanHuong' '162000186_NguyenVanHuong']


In [7]:
x.shape

(4508, 1, 128)

In [8]:
y.shape

(4508,)

In [9]:
x = x.reshape(-1, 128)

In [10]:
x.shape

(4508, 128)

In [11]:
# chia nhỏ dữ liệu
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.9, random_state=1)

In [13]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((4057, 128), (451, 128), (4057,), (451,))

In [14]:
# 2. Train machine learning

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [16]:
def get_report(model, x_train, y_train, x_test, y_test):
  y_pred_train = model.predict(x_train)
  y_pred_test = model.predict(x_test)

  # accuracy score
  acc_train = accuracy_score(y_train, y_pred_train)
  acc_test = accuracy_score(y_test, y_pred_test)

  # f1 score
  f1_score_train = f1_score(y_train, y_pred_train, average="macro")
  f1_score_test = f1_score(y_test, y_pred_test, average="macro")

  # print
  print('Accurency Train = %0.2f'%acc_train)
  print('Accurency Test = %0.2f'%acc_test)
  print('F1 Score Train = %0.2f'%f1_score_train)
  print('F1 Score Test = %0.2f'%f1_score_test)

In [17]:
# Logistic regression
model_logistic = LogisticRegression()
model_logistic.fit(x_train, y_train) # training

LogisticRegression()

In [18]:
get_report(model_logistic, x_train, y_train, x_test, y_test)

Accurency Train = 0.72
Accurency Test = 0.68
F1 Score Train = 0.52
F1 Score Test = 0.50


In [19]:
# Support Vector Machines
model_svc = SVC(probability=True)
model_svc.fit(x_train, y_train)

SVC(probability=True)

In [20]:
get_report(model_svc, x_train, y_train, x_test, y_test)

Accurency Train = 0.91
Accurency Test = 0.85
F1 Score Train = 0.84
F1 Score Test = 0.74


In [21]:
# Random forest
model_rf = RandomForestClassifier(n_estimators=20,)
model_rf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=20)

In [22]:
get_report(model_rf, x_train, y_train, x_test, y_test)

Accurency Train = 1.00
Accurency Test = 0.76
F1 Score Train = 1.00
F1 Score Test = 0.65


In [23]:
# Voting Classifier
model_voting = VotingClassifier(estimators=[
    ('logistic', LogisticRegression()),
    ('svm', SVC(probability=True)),
    ('rf', RandomForestClassifier(n_estimators=20,)),
], voting='soft', weights=[2,3,1])

In [24]:
model_voting.fit(x_train, y_train)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier(n_estimators=20))],
                 voting='soft', weights=[2, 3, 1])

In [25]:
get_report(model_voting, x_train, y_train, x_test, y_test)

Accurency Train = 0.98
Accurency Test = 0.85
F1 Score Train = 0.97
F1 Score Test = 0.74


In [26]:
# 3. Parameter Tuning
from sklearn.model_selection import GridSearchCV

model_grid = GridSearchCV(model_voting, param_grid={
    'svm__C': [3,5,7,10],
    'svm__gamma': [0.1, 0.3, 0.5],
    'rf__n_estimators': [5, 10,20],
    'rf__max_depth': [3, 5, 7],
    'voting': ['soft', 'hard']
}, scoring='accuracy', cv=3, n_jobs=1, verbose=2)

In [27]:
model_grid.fit(x_train, y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=  11.3s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   8.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   6.8s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   7.3s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   6.7s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   6.6s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   7.7s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   6.1s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   8.3s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gam

GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logistic',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier(n_estimators=20))],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [28]:
model_best_estimator = model_grid.best_estimator_

In [29]:
model_grid.best_score_

0.8461866463742637

In [30]:
# 4. Save Model
pickle.dump(model_best_estimator, open("/content/drive/MyDrive/TGMT/ml_face_person_identity.pkl", mode='wb'))